In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

Inception_Time

In [4]:
class InceptionModule(tf.keras.layers.Layer):
    def __init__(self,num_filters=32,activation='relu',**kwargs):
        super().__init__(**kwargs)
        self.num_filters=num_filters
        self.activation=tf.keras.activations.get(activation)
    
    def _default_Conv1D(self,filters,kernel_size):
        return tf.keras.layers.Conv1D(filters=filters,
                                     kernel_size=kernel_size,
                                     strides=1,
                                     activation='relu',
                                     use_bias=False)
    
    def call(self,inputs):
        z_bottleneck=self._default_Conv1D(filters=self.num_filters,kernel_size=1)(inputs)
        z_maxpool=tf.keras.layers.MaxPool1D(pool_size=3,strides=1,padding='same')(inputs)
    
        z1=self._default_Conv1D(filters=self.num_filters,kernel_size=1)(z_bottleneck)
        z2=self._default_Conv1D(filters=self.num_filters,kernel_size=1)(z_bottleneck)
        z3=self._default_Conv1D(filters=self.num_filters,kernel_size=1)(z_bottleneck)
        z4=self._default_Conv1D(filters=self.num_filters,kernel_size=1)(z_maxpool)
        
        z=tf.keras.layers.Concatenate(axis=2)([z1,z2,z3,z4])
        z=tf.keras.layers.BatchNormalization()(z)
        activation=self.activation(z)
        
        return activation

In [5]:
def shortcut_layer(inputs,z_inception):
    z_shortcut=tf.keras.layers.Conv1D(filters=int(out_tensorshape[-1],kernel_size=1,
                                              padding='same',use_bias='False'))(inputs)
    z_shortcut=tf.keras.layers.BatchNormalization()(z_shortcut)
    z=tf.keras.layers.Add()([z_shortcut,z_inception])
    
    return tf.keras.layers.Activation('relu')(Z)

def build_model(input_shape,num_classes,num_modules):
    input_layer=tf.keras.layers.Input(input_shape)
    z=input_layer
    z_residual=input_layer
    
    for i in range(num_modules):
        z=InceptionModule()(z)
        if i%3==2:
            z=shortcut_layer(z_residual,z)
            z_residucal=z
    
    gap_layer=tf.keras.layers.GlobalAveragePooling1D()(z)
    output_layer=tf.keras.layers.Dense(num_classes,activation='softmax')(gap_layer)
    
    model=tf.keras.models.Model(inputs=input_layer,outputs=output_layer)
    model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])
    
    return model

In [ ]:
model=build_model((4,4),3,1)

Multi-Layer_Perceptron

In [7]:
# n_input=X_final.shape[1]*X_final.shape[2]
# X_final=X_final.reshape((X_final.shape[0],n_input))
model=Sequential()
model.add(Dense(100,activation='relu',input_dim=n_input))
model.add(Dense(3))
model.compile(optimizer='adam',loss='mse',metrics=['accuracy'])